![title](./Pics/QLCon2018Banner.JPG)

# Creating Custom Visualizations in Power BI using ggplot2

### What is ggplot2

The ggplot2 package is a R package used to create custom visualizations. It is based on the *layered grammar of graphics*.

- **Aesthetic (aes)**: describes how the variables (columns) in your data are mapped to aesthetic attributes such as position, size, color, ect. The mapping can be done in the ggplot function or in the geom. If done in the ggplot function it will be available to all layers if done in the geom it will only be available for that layer.  
    
- **Layers**: geometric elements and statistical transformation.
    
- **Scales**: maps values in the data space to values in an aesthetic space

- **Themes**: Controls the non-data elements of your chart. Examples include things like chart title, background colors, font size, and ect.

First we need to load the R packages we will use to build the visualization. We will use the following 5 packages to build our visualization:

- **tidyverse**:  The tidyverse package is a combination of packages that shares the same API design and they are designed to make doing data science easier. We will be using the readr package to read in data, the dplyr package to reshape the data, and the ggplot2 package to create the visualization.

- **ggrepel**:  Sometimes when you plot data your data labels overlap. The ggrepel package reposition overlapping labels so that they are easier to read.

- **ggthemes**:  This package contains themes that can be used to format your non-data elements based on pre-defined styles

- **scales**: This package has multiple uses but we will be using it to rescale our data using the *rescale* function

- **stringi**: The stringi package contains many functions that you can use for string manipulation

In [ ]:
library(tidyverse)
library(ggrepel)
library(ggthemes)
library(scales)
library(stringi)

The code below reads in our data from a flat file to a R data frame.

In [ ]:
url <- "https://raw.githubusercontent.com/DieselAnalytics/PASS2018_PowerBI_GGPLOT2/master/Data/dataset.csv"
dataset <- read_csv(url)
head(dataset)

We are developing a quad chart for the NFL prospects based on the bench press and 40 yard dash. In order to do so the players in the quard chart must have bench press and 40 yard information. That information also needs to be on the same scale. The code below uses the *dplyr* package from *tidyverse* to reshape our data and the *scales* package to rescale our data.

In [ ]:
graph_data <-
    dataset %>%
    filter(`40YD` != 0 & BenchReps != 0) %>%
    mutate(
        Scaled.40YD = 
            round(rescale(`40YD` * -1, to = c(-10, 10)), 1)
        ,Scaled.BenchReps = 
            round(rescale(BenchReps, to = c(-10, 10)), 1)
    )
head(graph_data)

When we pass our data to R from Power BI our data needs to be in the proper shape for our visualization. When need to perform a test to make sure that is the case and handle situations when it is not. Below is a template that I use to make sure my data is in the correct shape.

### Logic template for a custom R visual for Power BI


        if (< dataset test code >) {

            # <Put code to generate graph here>
    
        } else {
    
            p <- ggplot(dataset, aes(x = BenchReps, y = `40YD`)) +
                 geom_blank() +
                 scale_x_continuous(
                      labels = NULL
                    , breaks = NULL
                 ) +
                 scale_y_continuous(
                      labels = NULL
                    , breaks = NULL
                 ) +
                 labs(x = NULL, y = NULL) +
                 theme_classic()
            p
        }

We know that our data is in the correct shape so now we can start building our visualization. We will do so in the same manner that Bob Ross used to do when he did his paintings. We will start with the chart then add layers until we get to the chart that we want.

![title](./Pics/Bob_at_Easel.jpg)

In [ ]:
p <- ggplot(graph_data, aes(x = Scaled.BenchReps, y = Scaled.40YD, col = Position)) +
     geom_point() +
     geom_label_repel(aes(label = Player), size = 4, show.legend = FALSE) +
     geom_hline(yintercept = 0) +
     geom_vline(xintercept = 0) + 
     labs(
         title = "My Combine Quad Chart"
     )
p

The *bquote* function in R allows you to combine strings with mathematical function. Below is a hack that I used to append the arrows to the axis titles.

In [ ]:
p <- p +
     xlab(bquote("Strength" ~ symbol('\256'))) +
     ylab(bquote("Speed" ~ symbol('\256')))
p

Code below labels our quadrants

In [ ]:
p <- p +
     # quad labels
     annotate("text", x = -5, y = -11, label = "Average", alpha = 0.2, size = 6) +
     annotate("text", x = -5, y = 11, label = "Fast", alpha = 0.2, size = 6) +
     annotate("text", x = 5, y = -11, label = "Strong", alpha = 0.2, size = 6) +
     annotate("text", x = 5, y = 11, label = "Beast Mode", alpha = 0.2, size = 6)
p

The code below draws a rectangular border around our labels

In [ ]:
p <- p +
     annotate("rect", xmin = -3.5, xmax = -6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
     annotate("rect", xmin = -3.5, xmax = -6.5, ymin = 10.5, ymax = 11.5, alpha = .2) +
     annotate("rect", xmin = 3.5, xmax = 6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
     annotate("rect", xmin = 3.5, xmax = 6.5, ymin = 10.5, ymax = 11.5, alpha = .2)
p

We apply formatting to our chart based on the principals developed by the renowned data visualization expert, [Edward Tufte](https://bit.ly/2s0WGc9).

In [ ]:
p <- p +
     theme_tufte()
p

We color quadrants 2 & 3

In [ ]:
p <- p +
     annotate("rect", xmin = -Inf, xmax = 0.0, ymin = -Inf, ymax = 0, alpha = 0.1, fill = "lightskyblue") +
     annotate("rect", xmin = 0.0, xmax = Inf, ymin = 0.0, ymax = Inf, alpha = 0.1, fill = "lightskyblue")
p

We now have the basics of our graph built. Now we need to add so dynamic information to our graph to give the end user important information. We will add a dynamic title, dynamic subtitle, and a caption that will give the end user the source of our data set. Before we start building the dynamic elements let's take a look at our data set.

In [ ]:
head(dataset)

Dynamically builds our chart title

In [ ]:
selected_position <- unique(dataset$`Position Group`)
selected_year <- unique(dataset$Year)
chart.title <- paste(selected_year, "Combine Quad Chart for", selected_position, sep = " ")
chart.title

Our dynamic subtitle will show up to 7 bullet points. The categories are:
- Players that did not run the 40 yard dash
- Players that did not perform the bench press
- Players that did not perform the bench press and 40 yard dash
- Players that did the best in the bench press
- Players that did the worse in the bench press
- Players that did the best in the 40 yard dash
- Players that did the worse in the 40 yard dash

Below is the code used to build the strings of the players that did not participate in the 40 yard dash, bench press, or both.

Subset the dataframe to only include the *Player* variable that did not run the 40 but did the bench press

In [ ]:
No40YDSubtitleHaveAthletes <-
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]
No40YDSubtitleHaveAthletes

Converts the variable fom above to a character vector

In [ ]:
No40YDSubtitle <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)] %>%
    as.character()
No40YDSubtitle

Converts the variable from above from a character vector to a string with each element separated by a *comma*

In [ ]:
No40YDSubtitle <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)] %>%
    as.character() %>%
    paste(collapse = ", ")
No40YDSubtitle

Replace the last comma in the string with *", and"*

In [ ]:
No40YDSubtitle <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)] %>%
    as.character() %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ")
No40YDSubtitle

Use the paste0 function to append a "- " to the beginning of the string from the cell above

In [ ]:
No40YDSubtitle <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)] %>%
    as.character() %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not run the 40YD.")
No40YDSubtitle

Passes the results from what was created in the cell above to the str_wrap function. The str_wrap function allows you to control the width of the string and also control the indentations from subsequent lines.

In [ ]:
No40YDSubtitle <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)] %>%
    as.character() %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not run the 40YD.") %>%
    str_wrap(width = 70, exdent = 2)
No40YDSubtitle

Do the same for the other "No" categories and creates variables that have the counts for each category.

In [ ]:
No40YDSubtitleHaveAthletes <-
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)] %>%
    length() %>%
    {ifelse(. > 0, "Y", "N")}

NoBenchPressSubtitle <- 
    dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)] %>%
    as.character() %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not participate in the benchpress.") %>%
    str_wrap(width = 70, exdent = 2)

NoBenchPressSubtitleHaveAthletes <- 
    dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)] %>%
    length() %>%
    {ifelse(. > 0, "Y", "N")}

No40YDAndBenchPressSubtitle <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)] %>%
    as.character() %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ") %>%
    paste0("- ", .," did not participate in either the 40YD or in the benchpress.") %>%
    str_wrap(width = 70, exdent = 2)

No40YDAndBenchPressSubtitleHaveAthletes <- 
    dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)] %>%
    length() %>%
    {ifelse(. > 0, "Y", "N")}

In [ ]:
No40YDSubtitle
NoBenchPressSubtitle
No40YDAndBenchPressSubtitle

The code cells below builds the subtitles for best benchpress, worst bench press, best 40 yard dash, and worst 40 yard dash.

In [ ]:
BestBench <- max(dataset$BenchReps)

BestBenchAthletes_cv <-
    dataset$Player[which(dataset$BenchReps == BestBench)] %>%
    as.character()
    
BestBenchAthletesCount <- length(BestBenchAthletes_cv)

BestBenchAthletes <- 
    BestBenchAthletes_cv %>%
    paste(collapse = ", ") %>%
    stri_replace_last_fixed(", ", ", and ")

BestBenchSubtitle <- 
    paste0(
        "- The highest number of reps for the benchpress was ",
        BestBench,
        ". ",
        BestBenchAthletes,
        ifelse(BestBenchAthletesCount > 1, " are ", " is "),
        "in this group"
    ) %>%
    str_wrap(width = 70, exdent = 2)

In [ ]:
    WorstBench <- min(dataset$BenchReps[dataset$BenchReps != 0])    

    WorstBenchAthletes_cv <- 
        dataset$Player[which(dataset$BenchReps == WorstBench)] %>%
        as.character()

    WorstBenchAthletesCount <- length(WorstBenchAthletes_cv)

    WorstBenchAthletes <- 
        WorstBenchAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")

    WorstBenchSubtitle <- 
        paste0(
            "- The lowest number of reps for the benchpress was ",
            WorstBench,
            ". ",
            WorstBenchAthletes,
            ifelse(WorstBenchAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 70, exdent = 2)


In [ ]:
    Best40YD <- min(dataset$`40YD`[dataset$`40YD` != 0])

    Best40YDAthletes_cv <- 
        dataset$Player[which(dataset$`40YD` == Best40YD)] %>%
        as.character()

    Best40YDAthletesCount <- length(Best40YDAthletes_cv)

    Best40YDAthletes <- 
        Best40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")

    Best40YDSubtitle <- 
        paste0(
            "- The fastest time for the 40YD was ",
            Best40YD,
            ". ",
            Best40YDAthletes,
            ifelse(Best40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 70, exdent = 2)

In [ ]:
    Worst40YD <- max(dataset$`40YD`)

    Worst40YDAthletes_cv <- 
        dataset$Player[which(dataset$`40YD` == Worst40YD)] %>%
        as.character()

    Worst40YDAthletesCount <- length(Worst40YDAthletes_cv)

    Worst40YDAthletes <- 
        Worst40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")

    Worst40YDSubtitle <- 
        paste0(
            "- The slowest time for the 40YD was ",
            Worst40YD,
            ". ",
            Worst40YDAthletes,
            ifelse(Worst40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 70, exdent = 2)

Combines all the subtitles elements from above to build the chart title

In [ ]:
    chartsubtitle <-
            paste0(
                ifelse(BestBenchAthletesCount == 0,"",paste0("\n", BestBenchSubtitle)),
                ifelse(WorstBenchAthletesCount == 0,"",paste0("\n", WorstBenchSubtitle)),
                ifelse(Best40YDAthletesCount == 0,"",paste0("\n", Best40YDSubtitle)),
                ifelse(Worst40YDAthletesCount == 0,"",paste0("\n", Worst40YDSubtitle)),
                ifelse(No40YDSubtitleHaveAthletes == "Y",paste0("\n", No40YDSubtitle),""),
                ifelse(NoBenchPressSubtitleHaveAthletes == "Y",paste0("\n", NoBenchPressSubtitle),""),
                ifelse(No40YDAndBenchPressSubtitleHaveAthletes == "Y",paste0("\n", No40YDAndBenchPressSubtitle),"")
            )
    chartsubtitle

In [ ]:
    chart_source <- "Source:  https://www.pro-football-reference.com"

What the chart looks like before the titles, subtitles, and caption

In [ ]:
p

What the chart looks like with the title, subtitle, and caption via the *labs* function

In [ ]:
p <- p +
     labs(
         title = chart.title, 
         subtitle = chartsubtitle, 
         caption = chart_source
     )
p

In [ ]:
p <- p +
     theme(
          plot.title = element_text(hjust = 0.5, size = 25)
        , plot.subtitle = element_text(size = 15) 
        , panel.border = element_rect(colour = "black", size = 2, fill = NA)
        , axis.title.x = element_text(hjust = 0.1, size = 18)
        , axis.title.y = element_text(hjust = 0.1, size = 18)
     )
p

In [ ]:
library(tidyverse)
library(ggrepel)
library(ggthemes)
library(scales)
library(stringi)

if (length(unique(dataset$Year)) == 1 & length(unique(dataset$`Position Group`)) == 1) {
  
    selected_position <- unique(dataset$`Position Group`)
    chart.title <- paste(min(dataset$Year), "Combine Quad Chart for", selected_position, sep = " ")

    No40YDSubtitle <- 
        as.character(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]) %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ") %>%
        paste0("- ", .," did not run the 40YD.") %>%
        str_wrap(width = 140, exdent = 2)
    No40YDSubtitleHaveAthletes <-
        length(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps != 0)]) %>%
        {ifelse(. > 0, "Y", "N")}
    
    NoBenchPressSubtitle <- 
        as.character(dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)]) %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ") %>%
        paste0("- ", .," did not participate in the benchpress.") %>%
        str_wrap(width = 140, exdent = 2)
    NoBenchPressSubtitleHaveAthletes <- 
        length(dataset$Player[which(dataset$`40YD` != 0 & dataset$BenchReps == 0)]) %>%
        {ifelse(. > 0, "Y", "N")}
    
    No40YDAndBenchPressSubtitle <- 
        as.character(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)]) %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ") %>%
        paste0("- ", .," did not participate in either the 40YD or in the benchpress.") %>%
        str_wrap(width = 140, exdent = 2)
    No40YDAndBenchPressSubtitleHaveAthletes <- 
        length(dataset$Player[which(dataset$`40YD` == 0 & dataset$BenchReps == 0)]) %>%
        {ifelse(. > 0, "Y", "N")}

    graph_data <-
        dataset %>%
        filter(`40YD` != 0 & BenchReps != 0) %>%
        mutate(
            Scaled.40YD = 
                round(rescale(`40YD` * -1, to = c(-10, 10)), 1)
            ,Scaled.BenchReps = 
                round(rescale(BenchReps, to = c(-10, 10)), 1)
        )

    BestBench <- max(dataset$BenchReps)
    BestBenchAthletes_cv <- as.character(dataset$Player[which(dataset$BenchReps == BestBench)])
    BestBenchAthletesCount <- length(BestBenchAthletes_cv)
    BestBenchAthletes <- 
        BestBenchAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    BestBenchSubtitle <- 
        paste0(
            "- The highest number of reps for the benchpress was ",
            BestBench,
            ". ",
            BestBenchAthletes,
            ifelse(BestBenchAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)

    WorstBench <- min(dataset$BenchReps[dataset$BenchReps != 0])   
    WorstBenchAthletes_cv <- as.character(dataset$Player[which(dataset$BenchReps == WorstBench)])
    WorstBenchAthletesCount <- length(WorstBenchAthletes_cv)
    WorstBenchAthletes <- 
        WorstBenchAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    WorstBenchSubtitle <- 
        paste0(
            "- The lowest number of reps for the benchpress was ",
            WorstBench,
            ". ",
            WorstBenchAthletes,
            ifelse(WorstBenchAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)

    Best40YD <- min(dataset$`40YD`[dataset$`40YD` != 0])
    Best40YDAthletes_cv <- as.character(dataset$Player[which(dataset$`40YD` == Best40YD)])
    Best40YDAthletesCount <- length(Best40YDAthletes_cv)
    Best40YDAthletes <- 
        Best40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    Best40YDSubtitle <- 
        paste0(
            "- The fastest time for the 40YD was ",
            Best40YD,
            ". ",
            Best40YDAthletes,
            ifelse(Best40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)

    Worst40YD <- max(dataset$`40YD`)
    Worst40YDAthletes_cv <- as.character(dataset$Player[which(dataset$`40YD` == Worst40YD)])
    Worst40YDAthletesCount <- length(Worst40YDAthletes_cv)
    Worst40YDAthletes <- 
        Worst40YDAthletes_cv %>%
        paste(collapse = ", ") %>%
        stri_replace_last_fixed(", ", ", and ")
    Worst40YDSubtitle <- 
        paste0(
            "- The slowest time for the 40YD was ",
            Worst40YD,
            ". ",
            Worst40YDAthletes,
            ifelse(Worst40YDAthletesCount > 1, " are ", " is "),
            "in this group"
        ) %>%
        str_wrap(width = 140, exdent = 2)
    
    chartsubtitle <-
            paste0(
                ifelse(BestBenchAthletesCount == 0,"",paste0("\n", BestBenchSubtitle)),
                ifelse(WorstBenchAthletesCount == 0,"",paste0("\n", WorstBenchSubtitle)),
                ifelse(Best40YDAthletesCount == 0,"",paste0("\n", Best40YDSubtitle)),
                ifelse(Worst40YDAthletesCount == 0,"",paste0("\n", Worst40YDSubtitle)),
                ifelse(No40YDSubtitleHaveAthletes == "Y",paste0("\n", No40YDSubtitle),""),
                ifelse(NoBenchPressSubtitleHaveAthletes == "Y",paste0("\n", NoBenchPressSubtitle),""),
                ifelse(No40YDAndBenchPressSubtitleHaveAthletes == "Y",paste0("\n", No40YDAndBenchPressSubtitle),"")
            )
    
    chart_source <- "Source:  https://www.pro-football-reference.com"

    p <- ggplot(graph_data, aes(x = Scaled.BenchReps, y = Scaled.40YD, col = Position)) +
         geom_point() +
         geom_label_repel(aes(label = Player), size = 4, show.legend = FALSE) +
         geom_hline(yintercept = 0) +
         geom_vline(xintercept = 0) +

         # quad labels
         annotate("text", x = -5, y = -11, label = "Average", alpha = 0.2, size = 6) +
         annotate("text", x = -5, y = 11, label = "Fast", alpha = 0.2, size = 6) +
         annotate("text", x = 5, y = -11, label = "Strong", alpha = 0.2, size = 6) +
         annotate("text", x = 5, y = 11, label = "Beast Mode", alpha = 0.2, size = 6) +

         # Squares for quad labels
         annotate("rect", xmin = -3.5, xmax = -6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
         annotate("rect", xmin = -3.5, xmax = -6.5, ymin = 10.5, ymax = 11.5, alpha = .2) +
         annotate("rect", xmin = 3.5, xmax = 6.5, ymin = -11.5, ymax = -10.5, alpha = .2) +
         annotate("rect", xmin = 3.5, xmax = 6.5, ymin = 10.5, ymax = 11.5, alpha = .2) +

         # Shade lower left quadrant
         annotate("rect", xmin = -Inf, xmax = 0.0, ymin = -Inf, ymax = 0, alpha = 0.1, fill = "lightskyblue") +
        
         # Shade upper right quadrant
         annotate("rect", xmin = 0.0, xmax = Inf, ymin = 0.0, ymax = Inf, alpha = 0.1, fill = "lightskyblue") +
        
         # Titles
         xlab(bquote("Strength" ~ symbol('\256'))) +
         ylab(bquote("Speed" ~ symbol('\256'))) +
         #ggtitle(chart.title, subtitle = chartsubtitle) +
         labs(title = chart.title, subtitle = chartsubtitle, caption = chart_source) +

         # Prettying things up
         theme_tufte() +
         theme(
              plot.title = element_text(hjust = 0.5, size = 25)
            , plot.subtitle = element_text(size = 15) 
            , panel.border = element_rect(colour = "black", size = 2, fill = NA)
            , axis.title.x = element_text(hjust = 0.1, size = 18)
            , axis.title.y = element_text(hjust = 0.1, size = 18)
         ) +
         scale_x_continuous(labels = NULL, breaks = NULL) +
         scale_y_continuous(labels = NULL, breaks = NULL) 
    
         p

} else {
    p <- ggplot(dataset, aes(x = BenchReps, y = `40YD`)) +
         geom_blank() +
         scale_x_continuous(
              labels = NULL
            , breaks = NULL
        ) +
         scale_y_continuous(
              labels = NULL
            , breaks = NULL
         ) +
         labs(x = NULL, y = NULL) +
         theme_classic()
    p
}